# Examples of usage

In [1]:
import numpy as np
import logging

In [2]:
logging.basicConfig()
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

In [3]:
from core import WMFileSource, PythonSource, MNPSolver
from mnp import (GurobiAlgorithm, GreedyAlgorithm, SimulatedAnnealing,
                transition_greedy_n, transition_greedy_one, temperature_div,
                temperature_div_log, transition_greedy_many, KarmarkarKarp)


PartialSet 140483009389184
sets: [[3, 1, 8, 6], [4, 7, 2, 5]],
sums: [0, 0],
error: 0.
PartialSet 140483009389760
sets: [[6, 5, 8, 3], [7, 4, 9, 2, 1]],
sums: [0, 1],
error: 1.
PartialSet 140483009389808
sets: [[7, 4], [8, 3], [9, 2], [1, 6, 5]],
sums: [0, 0, 0, 1],
error: 1.
PartialSet 140483009049024
sets: [[5250, 3430, 7105, 5348, 2996, 9702, 7301, 2114, 3927, 5586, 8918, 8239, 6965, 1323, 6811, 1379, 8631, 4018, 6272, 6083, 3241, 441, 5047, 1085, 5446, 1617, 4459, 3290, 3633, 7056, 4466, 7840, 1722, 3822, 7406, 1176, 1820, 140, 4368, 4361, 6426, 6223, 8484, 8085, 4711, 9065, 448, 4312, 5537, 9366, 3871, 7896, 3528, 595, 4998, 497, 6664, 2898, 2751, 4802, 4613, 2891, 3192, 6076, 5985, 8428, 5789, 9653, 3094, 4116, 7742, 5054, 8680, 5733, 4704, 1526, 9464, 5439, 1470, 2016, 2212, 6419, 9114, 2457, 6867, 882, 4606, 2604, 350, 2205, 8330, 9268, 3437, 7595, 9898, 2702, 7994, 7987, 3185, 2555, 8624, 9709, 9359, 6916, 5243, 2359, 2303, 9562, 5782, 1183, 7350, 6328, 1666, 7651, 7252, 4025,

# Small Instance

In [4]:
ext_s_one = PythonSource(
    data=[8, 6, 7, 4, 5],
    mapper=float,
    gatherer=list,
)

## Greedy Algorithm

In [5]:
solver = MNPSolver(source=ext_s_one, algorithm=GreedyAlgorithm(number_of_sets=3))
greedy_of, greedy_sets = solver.solve_problem()
solver

MNP SOLVER INSTANCE
SOURCE: <core.data_source.PythonSource object at 0x7f2ce9e53e20>
ALGORITHM: <mnp.greedy.GreedyAlgorithm object at 0x7f2d29c905e0>
OF: 6.0
SETS:
SET #0: [8.0]
SET #1: [7.0, 4.0]
SET #2: [6.0, 5.0]
SET SUMS: 8.0 11.0 11.0

## Simulated Annealing

In [6]:
solver = MNPSolver(
    source=ext_s_one,
    algorithm=SimulatedAnnealing(
        start_partitioning=[[8, 7, 4, 6, 5], [], []],
        transition_func=transition_greedy_n(7),
        temperature_func=temperature_div_log,
        t_max=10**6,
        t_min=10**-12,
        max_iter=10**2,
        number_of_sets=3,
    )
)
_of, _sets = solver.solve_problem()
solver

MNP SOLVER INSTANCE
SOURCE: <core.data_source.PythonSource object at 0x7f2ce9e53e20>
ALGORITHM: <mnp.simulated_annealing.SimulatedAnnealing object at 0x7f2d29c90ca0>
OF: 6.0
SETS:
SET #0: [8]
SET #1: [4, 7]
SET #2: [6, 5]
SET SUMS: 8 11 11

## Gurobi

In [7]:
solver = MNPSolver(source=ext_s_one, algorithm=GurobiAlgorithm(number_of_sets=3))
_of, _sets = solver.solve_problem()
solver

Restricted license - for non-production use only - expires 2022-01-13
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 5 rows, 15 columns and 15 nonzeros
Model fingerprint: 0x255c147a
Model has 45 quadratic objective terms
Variable types: 0 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e+01, 2e+02]
  QObjective range [3e+01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 600.0000000
Presolve time: 0.00s
Presolved: 35 rows, 45 columns, 105 nonzeros
Variable types: 0 continuous, 45 integer (45 binary)

Root relaxation: objective -1.100000e+02, 11 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 -110.00000    0    8 

MNP SOLVER INSTANCE
SOURCE: <core.data_source.PythonSource object at 0x7f2ce9e53e20>
ALGORITHM: <mnp.exact.GurobiAlgorithm object at 0x7f2ce9e68610>
OF: 6.0
SETS:
SET #0: [7.0, 4.0]
SET #1: [6.0, 5.0]
SET #2: [8.0]
SET SUMS: 11.0 11.0 8.0

## Karmarkar

In [5]:
KarmarkarKarp(ext_s_one.get_data(), m=3)


PartialSet 140483008723696
sets: [[8.0], [4.0, 7.0], [5.0, 6.0]],
sums: [0.0, 3.0, 3.0],
error: 6.0.

# Big instance

In [6]:
wms = WMFileSource(
    filepath='instances/data.wm',
    mapper=int,
    gatherer=list,
)

## Greedy Algorithm

In [9]:
solver = MNPSolver(source=wms, algorithm=GreedyAlgorithm(number_of_sets=20))
greedy_of, greedy_sets = solver.solve_problem()
solver

MNP SOLVER INSTANCE
SOURCE: <core.data_source.WMFileSource object at 0x7f2ce9e68580>
ALGORITHM: <mnp.greedy.GreedyAlgorithm object at 0x7f2ce9e68400>
OF: 7612188.0
SETS:
SET #0: [387162, 201347, 89243, 81049, 69547, 60230, 51051, 45444, 30247, 23256, 16676]
SET #1: [375698, 206707, 89859, 81724, 71640, 65415, 50152, 38229, 37140, 21309, 18793]
SET #2: [372906, 208596, 90314, 82469, 70866, 65893, 49903, 38230, 38206, 20440, 19311]
SET #3: [357826, 210543, 95292, 88151, 71789, 68027, 49046, 40868, 34194, 23080, 14485, 1933, 1302]
SET #4: [349541, 211791, 99148, 88870, 80539, 55712, 53662, 46912, 27698, 26581, 9948, 6415]
SET #5: [346671, 219218, 98418, 86760, 78372, 56643, 52539, 47239, 27707, 27196, 8993, 7059]
SET #6: [342797, 228372, 92616, 87745, 73494, 66449, 49418, 40264, 34999, 22385, 16959]
SET #7: [339311, 234039, 92018, 84625, 78777, 59258, 51914, 44558, 30807, 23523, 14191, 3606]
SET #8: [339195, 240534, 91393, 83809, 69933, 66543, 49526, 39093, 35133, 25408, 9152, 7314]
SET #

## Simulated Annealing

In [10]:
solver = MNPSolver(
    source=wms,
    algorithm=SimulatedAnnealing(
        start_partitioning=greedy_sets,
        transition_func=transition_greedy_many(5),
        temperature_func=temperature_div_log,
        t_max=10**5,
        t_min=0,
        max_iter=10**5,
        number_of_sets=20,
    )
)
_of, _sets = solver.solve_problem()
solver

MNP SOLVER INSTANCE
SOURCE: <core.data_source.WMFileSource object at 0x7f2ce9e68580>
ALGORITHM: <mnp.simulated_annealing.SimulatedAnnealing object at 0x7f2ce9dfd9a0>
OF: 3682988.0
SETS:
SET #0: [70866, 16676, 23181, 201347, 89243, 45444, 387162, 51051, 81049, 30247, 60230]
SET #1: [50152, 81724, 37140, 21309, 38206, 206707, 65415, 375698, 18793, 89859, 71640]
SET #2: [39175, 69547, 49418, 372906, 90314, 82469, 19311, 65893, 38229, 208596, 20440]
SET #3: [1933, 71789, 1302, 14485, 88151, 34194, 40868, 95292, 357826, 210543, 22890, 49046, 68027]
SET #4: [55712, 99148, 27698, 9948, 88870, 349541, 211791, 6415, 53662, 26581, 46912, 80539]
SET #5: [27707, 86760, 8993, 78372, 346671, 52539, 27196, 98418, 47239, 56643, 219218, 7059]
SET #6: [40264, 342797, 16959, 87745, 92616, 73494, 66449, 34999, 22385, 228372, 50322]
SET #7: [234039, 59258, 92018, 51914, 339311, 23256, 3606, 14191, 84625, 78777, 44558, 30807]
SET #8: [49526, 83809, 7314, 66543, 25408, 339195, 38230, 69933, 35133, 91393, 915

## Tabu Search

## Ant Colony Optimization


## Genetic Algorithms

## Karmarkar

In [7]:
KarmarkarKarp(wms.get_data(), m=20)


PartialSet 140483009492928
sets: [[25623, 40391, 34940, 80457, 55712, 311060, 260107, 92820, 87539, 50472, 16676], [45444, 72297, 65415, 30137, 53662, 293515, 279671, 92359, 83946, 22890, 11501, 4967], [45268, 74335, 63483, 30247, 297393, 271928, 94830, 87100, 51893, 22385, 16959], [46232, 69547, 68027, 28470, 372906, 208596, 90314, 82469, 49418, 23523, 10360, 5999], [26308, 38230, 37140, 79350, 56643, 342797, 228372, 92616, 87745, 50322, 9948, 6415], [46912, 70866, 66543, 28215, 375698, 206707, 89859, 81724, 49526, 23256, 11812, 4830], [38229, 38206, 78777, 56716, 24948, 339311, 234039, 92018, 84625, 52539, 10588, 5979], [26581, 39175, 35129, 76620, 60230, 357826, 210543, 95292, 88151, 49903, 10373, 6276], [25873, 41794, 33906, 80539, 55124, 303036, 262574, 98472, 87268, 50579, 15515, 1502], [46999, 78134, 59258, 27707, 387162, 201347, 89243, 81049, 47520, 20440, 17355], [25408, 40264, 34999, 78406, 57889, 298160, 270015, 96607, 86551, 51051, 11015, 5968], [25811, 40868, 34194, 78372,

# Benchmarking

In [ ]:
# TODO: imports

In [ ]:
# TODO: Table Algorithm: (OF, Time)

In [ ]:
# TODO: Plot N/Time